In [ ]:
from deepar.dataset.time_series import TimeSeries
from deepar.model.lstm import DeepARLearner


In [ ]:
import pandas as pd
sunspots_df = pd.read_csv('../datasets/seed_datasets_current/56_sunspots/TRAIN/dataset_TRAIN/tables/learningData.csv')
sunspots_df['year'] = pd.to_timedelta(pd.to_datetime(sunspots_df['year'], format='%Y')).dt.total_seconds()
sunspots_ds_one = TimeSeries(sunspots_df, target_idx = 4, timestamp_idx = 1, index_col=0)

In [ ]:
from deepar.dataset.time_series import TimeSeries, TimeSeriesTest
import pandas as pd
import numpy as np
from deepar.model.learner import DeepARLearner

sunspots_df = pd.read_csv('../datasets/seed_datasets_current/56_sunspots/TRAIN/dataset_TRAIN/tables/learningData.csv')
# sunspots_df['sunspots'][5] = np.nan
# sunspots_df['sunspots'][10] = np.nan
# sunspots_df['sunspots'][15] = np.nan
# sunspots_df['sunspots'][20] = np.nan
# sunspots_df['sunspots'][25] = np.nan
sunspots_df['year'] = pd.to_timedelta(pd.to_datetime(sunspots_df['year'], format='%Y')).dt.total_seconds()
sunspots_ds_one = TimeSeries(sunspots_df, target_idx = 4, timestamp_idx = 1, index_col=0)

learner = DeepARLearner(sunspots_ds_one, verbose=1)
!rm -r ./tb_test
learner.fit(epochs = 1, batches = 10, early_stopping = False, checkpoint_dir = "./tb_test")

test_df = pd.read_csv('../datasets/seed_datasets_current/56_sunspots/TEST/dataset_TEST/tables/learningData.csv')
test_df['year'] = pd.to_timedelta(pd.to_datetime(test_df['year'], format='%Y')).dt.total_seconds()
test_ds = TimeSeriesTest(test_df, sunspots_ds_one, target_idx = 4, timestamp_idx = 1, index_col=0)

preds = learner.predict(test_ds, horizon = None, samples = 1, include_all_training = True).reshape(-1)
scores = pd.read_csv('../datasets/seed_datasets_current/56_sunspots/SCORE/dataset_SCORE/tables/learningData.csv')

# evaluate compared to D3M test set
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(scores['sunspots'], preds))
print(f'rms: {rms}')

In [ ]:
from deepar.dataset.time_series import TimeSeries, TimeSeriesTest
import pandas as pd
import numpy as np
from deepar.model.learner import DeepARLearner

pop_df = pd.read_csv('../datasets/seed_datasets_current/LL1_736_population_spawn/TRAIN/dataset_TRAIN/tables/learningData.csv')
pop_df = pop_df.set_index(['species', 'sector'])
pop_df['group'] = pop_df.index
pop_df['group'] = pop_df['group'].apply(lambda x: " ".join(x))
pop_df = pop_df.reset_index(drop=True)
pop_df['day'] = pd.to_timedelta(pd.to_datetime(pop_df['day'], unit='D')).dt.total_seconds()
pop_ds = TimeSeries(pop_df, target_idx = 2, timestamp_idx = 1, index_col=0, grouping_idx = 3, count_data = True)

learner = DeepARLearner(pop_ds, verbose = 1)
learner.fit(epochs = 1, batches = 10, early_stopping = False)

test_df = pd.read_csv('../datasets/seed_datasets_current/LL1_736_population_spawn/TEST/dataset_TEST/tables/learningData.csv')
test_df = test_df.set_index(['species', 'sector'])
test_df['group'] = test_df.index
test_df['group'] = test_df['group'].apply(lambda x: " ".join(x))
test_df = test_df.reset_index(drop=True)
test_df['day'] = pd.to_timedelta(pd.to_datetime(test_df['day'], unit='D')).dt.total_seconds()
test_ds = TimeSeriesTest(test_df, pop_ds, target_idx = 2, timestamp_idx = 1, index_col=0, grouping_idx = 3)

preds = learner.predict(test_ds, horizon = None, samples = 1, include_all_training = True)

# evaluate compared to D3M test set
scores = pd.read_csv('../datasets/seed_datasets_current/LL1_736_population_spawn/SCORE/dataset_SCORE/tables/learningData.csv')
from sklearn.metrics import mean_squared_error
from math import sqrt
print(scores['count'].shape)
# rms = sqrt(mean_squared_error(scores['count'], preds))
# print(f'rms: {rms}')



In [ ]:
%matplotlib inline
from numpy.random import normal
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
window = 20
features = 9
def get_sample_prediction(sample, fn):
    sample = np.array(sample).reshape(1, window, features)
    output = fn([sample])
    samples = []
    for mu,sigma in zip(output[0].reshape(window), output[1].reshape(window)):
        samples.append(normal(loc=mu, scale=np.sqrt(sigma), size=1)[0])
    return np.array(samples)

batch = sunspots_ds_one.next_batch(1, window)
ress = []
for i in range(300):
    ress.append(get_sample_prediction(batch[0], dp_model.predict_theta_from_input))

res_df = pd.DataFrame(ress).T
tot_res = res_df


plt.plot(batch[1].reshape(window), linewidth=6)
tot_res['mu'] = tot_res.apply(lambda x: np.mean(x), axis=1)
tot_res['upper'] = tot_res.apply(lambda x: np.mean(x) + np.std(x), axis=1)
tot_res['lower'] = tot_res.apply(lambda x: np.mean(x) - np.std(x), axis=1)
tot_res['two_upper'] = tot_res.apply(lambda x: np.mean(x) + 2*np.std(x), axis=1)
tot_res['two_lower'] = tot_res.apply(lambda x: np.mean(x) - 2*np.std(x), axis=1)


plt.plot(tot_res.mu, 'bo')
plt.plot(tot_res.mu, linewidth=2)
plt.fill_between(x = tot_res.index, y1=tot_res.lower, y2=tot_res.upper, alpha=0.5)
plt.fill_between(x = tot_res.index, y1=tot_res.two_lower, y2=tot_res.two_upper, alpha=0.5)
plt.title('Prediction uncertainty')
plt.show()


In [ ]:
from deepar.model.loss import negative_binomial_likelihood
from deepar.model.loss import gaussian_likelihood
import tensorflow as tf

sigma = 1.0
sigma1 = 2.0
sigma2 = 4.0
g = gaussian_likelihood([sigma] * 3)
b = negative_binomial_likelihood([sigma] * 3)
b1 = negative_binomial_likelihood([sigma1] * 3)
b2 = negative_binomial_likelihood([sigma2] * 3)

with tf.Session() as sess:
    y_true = tf.convert_to_tensor([1,1,1], dtype=tf.float32)
    y_pred0 = tf.convert_to_tensor([1,1,1], dtype=tf.float32)
    y_pred1 = tf.convert_to_tensor([2,2,2], dtype=tf.float32)
    y_pred2 = tf.convert_to_tensor([4,4,4], dtype=tf.float32)
    y_pred3 = tf.convert_to_tensor([8,8,8], dtype=tf.float32)
    print(b(y_true, y_pred0).eval())
    print(b(y_true, y_pred1).eval())
    print(b(y_true, y_pred2).eval())
    print(b(y_true, y_pred3).eval())
    print(b1(y_true, y_pred0).eval())
    print(b2(y_true, y_pred0).eval())

In [ ]:
x = pd.to_datetime("21/11/06 16:30", format="%d/%m/%y %H:%M")
x.timetuple().tm_yday

In [ ]:
df.groupby('cat').apply(lambda x: x.count())['cat']

In [ ]:
df.groupby('cat').sum().reset_index()

In [ ]:
for cat, c_d in df.groupby('cat'):
    c_d['age'] = c_d.index
    print(c_d)

In [ ]:
df = df.drop('age', axis=1)

In [ ]:
df['age'] = df.groupby('cat').cumcount()

In [ ]:
df['age1'] = range(df.shape[0])

In [ ]:
df

In [ ]:
from sklearn.preprocessing import StandardScaler
df[[0,1]] = StandardScaler().fit_transform(df[[0,1]])

In [ ]:
1 + df.groupby('cat')[0].agg('mean')